### Import libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.linalg import Vectors
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

### Create session

In [ ]:
spark = SparkSession.builder \
    .appName("MLlib") \
    .master("local[1]") \
    .config("spark.driver.memory", "1g") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 10:04:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Prepare data and preprocessing

In [ ]:
train_path = "train.csv"
test_path = "test.csv"
train_df = spark.read.csv(train_path, header=True, inferSchema=True)
test_df = spark.read.csv(test_path, header=True, inferSchema=True)

#### Remove outliers

In [4]:
quantiles = train_df.approxQuantile("trip_duration", [0.25, 0.75], 0.01)
Q1, Q3 = quantiles
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
train_df = train_df.filter((col("trip_duration") >= lower_bound) & (col("trip_duration") <= upper_bound))


In [ ]:
feature_cols = ["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]

def to_labeled_point(row):
    features = [row[c] for c in feature_cols]
    return LabeledPoint(row["trip_duration"], Vectors.dense(features))


In [ ]:
train_rdd = train_df.rdd.map(to_labeled_point)

### Split data to train and test

In [ ]:
train_data, val_data = train_rdd.randomSplit([0.8, 0.2], seed=42)

### Training model

In [8]:
model = DecisionTree.trainRegressor(
    train_data,
    categoricalFeaturesInfo={},
    impurity="variance",
    maxDepth=20
)

25/04/11 10:19:53 WARN DAGScheduler: Broadcasting large task binary with size 1496.8 KiB
25/04/11 10:19:53 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/04/11 10:19:54 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
25/04/11 10:19:55 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB
25/04/11 10:19:56 WARN DAGScheduler: Broadcasting large task binary with size 1069.0 KiB
25/04/11 10:19:57 WARN DAGScheduler: Broadcasting large task binary with size 8.0 MiB
25/04/11 10:19:57 WARN DAGScheduler: Broadcasting large task binary with size 1379.4 KiB
25/04/11 10:19:58 WARN DAGScheduler: Broadcasting large task binary with size 11.1 MiB
25/04/11 10:19:59 WARN DAGScheduler: Broadcasting large task binary with size 1652.8 KiB
25/04/11 10:20:00 WARN DAGScheduler: Broadcasting large task binary with size 14.6 MiB
25/04/11 10:20:02 WARN DAGScheduler: Broadcasting large task binary with size 1889.6 KiB


### Testing model

In [ ]:
predictions = model.predict(val_data.map(lambda x: x.features))
labels_and_preds = val_data.map(lambda lp: lp.label).zip(predictions)

metrics = RegressionMetrics(labels_and_preds)
print(f"RMSE: {metrics.rootMeanSquaredError:.6f}")
print(f"R²: {metrics.r2:.6f}")

/Users/macos/Desktop/hcmus/big_data/Lab-3-Big-Data/venv/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
25/04/11 10:20:07 WARN DAGScheduler: Broadcasting large task binary with size 16.1 MiB
25/04/11 10:20:47 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 50 (TID 95): Attempting to kill Python Worker
25/04/11 10:20:47 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 50 (TID 95): Attempting to kill Python Worker
25/04/11 10:20:47 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 50 (TID 95): Attempting to kill Python Worker
25/04/11 10:20:55 WARN DAGScheduler: Broadcasting large task binary with size 16.1 MiB


RMSE: 289.026082
R²: 0.367998


### Run on test file

In [ ]:
feature_cols_local = ["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]

test_rows = test_df.select("id", *feature_cols_local).collect()

results = []
for row in test_rows:
    id_val = row["id"]
    features = Vectors.dense([row[c] for c in feature_cols_local])
    prediction = float(model.predict(features))
    results.append((id_val, prediction))

schema = StructType([
    StructField("id", StringType(), True),
    StructField("prediction", DoubleType(), True)
])
predicted_df = spark.createDataFrame(results, schema=schema)

In [ ]:
predicted_df.toPandas().to_csv("results.csv", index=False)

25/04/11 10:47:39 WARN TaskSetManager: Stage 53 contains a task of very large size (14066 KiB). The maximum recommended task size is 1000 KiB.
